# Check the number of Cryptocurrencies available on Coinbase

In [7]:
import requests

# Define the API endpoint
url = 'https://api.pro.coinbase.com'

# Make a request to get a list of all available currency pairs
response = requests.get(f'{url}/products')

# Parse the response as JSON
pairs = response.json()

# Collect the currency pair IDs in a list
currency_pairs = [pair['id'] for pair in pairs]

# Create a comma-separated string of currency pairs
comma_separated_pairs = ', '.join(currency_pairs)

# Print the comma-separated currency pairs
print(comma_separated_pairs)


LOKA-USD, NU-BTC, XTZ-GBP, BTRST-GBP, MANA-BTC, ETH-USD, XLM-USDT, ILV-USD, SKL-EUR, PUNDIX-USD, PLU-USD, ADA-USDT, REQ-BTC, MONA-USD, RLC-BTC, BOBA-USD, ANKR-GBP, BNT-GBP, USDC-GBP, LINK-BTC, MANA-USD, CHZ-USD, API3-USDT, LTC-BTC, NEAR-USD, CTSI-BTC, WBTC-USD, POND-USD, IDEX-USD, ENS-USD, CLV-EUR, METIS-USD, USDT-USDC, MASK-EUR, UNI-BTC, RAD-EUR, BADGER-USDT, BADGER-EUR, ICP-USDT, HOPR-USD, ARPA-USDT, MATIC-GBP, REQ-USD, BTC-USDT, LCX-USD, MINA-EUR, CGLD-GBP, LINK-EUR, ETH-BTC, ERN-USDT, LOOM-USD, POWR-EUR, AURORA-USD, MIR-GBP, LCX-EUR, COMP-USD, OMG-GBP, BAND-EUR, PAX-USD, CVX-USD, LINK-ETH, OGN-BTC, CBETH-USD, AXL-USD, BCH-USD, ZEC-BTC, ADA-EUR, QSP-USD, SHPING-USD, GNO-USDT, XTZ-BTC, CRV-GBP, UST-USD, NU-USD, FLOW-USDT, TRIBE-USD, SNX-USD, ALGO-EUR, FORTH-EUR, WBTC-BTC, DASH-USD, RNDR-USD, REQ-GBP, PERP-USD, ADA-BTC, SOL-USDT, SHPING-USDT, MDT-USDT, FIL-GBP, NKN-USD, EGLD-USD, SOL-EUR, WCFG-BTC, APE-USDT, FORTH-GBP, NMR-BTC, REN-USD, UNI-USD, ETC-EUR, ICP-USD, LQTY-USDT, BCH-EUR, N

# Download each currency pair and calculate liquidity using the Index of Martin (1975).

In [1]:
import requests
import pandas as pd
import os

# Define the API endpoint
url = 'https://api.pro.coinbase.com'

# Define the path where the CSV files will be stored
data_folder = '/content/coinbase_data'

# Create the data folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Get a list of all currency pairs
pairs = requests.get(f'{url}/products').json()

# Define a function to download data for a given currency pair
def download_data(pair):
    # Define the data URL for the currency pair
    data_url = f'{url}/products/{pair}/candles?granularity=86400'
    
    # Download the data for the currency pair
    response = requests.get(data_url)
    
    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(response.json(), columns=['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Save the data to a CSV file
    df.to_csv(f'{data_folder}/{pair}.csv', index=False)
    print(f'Downloaded data for {pair}')
    
    # Calculate MLI for the currency pair
    mlis = []
    for i in range(1, len(df)):
        pt = df.iloc[i]['Close']
        pt_minus_1 = df.iloc[i-1]['Close']
        vt = df.iloc[i]['Volume']
        mli = ((pt - pt_minus_1)**2) / vt
        mlis.append(mli)
    mli_value = sum(mlis)
    volume = df['Volume'].sum()
    all_ml_values.append({'Currency Pair': pair, 'MLI': mli_value, 'Volume': volume})
    print(f"MLI for {pair}: {mli_value}")

# Download data for all currency pairs and calculate MLI
all_ml_values = []
for pair in pairs:
    download_data(pair['id'])
    
# Save the individual MLI values to a CSV file in the destination folder
if all_ml_values:
    pd.DataFrame(all_ml_values).to_csv(f'{data_folder}/individual_mli.csv', index=False)
    print('Saved individual MLI values to CSV')
else:
    print("No MLI values calculated due to errors")

# Calculate overall exchange MLI weighted with traded volume
if all_ml_values:
    df_all = pd.DataFrame(all_ml_values)
    total_volume = df_all.groupby(['Currency Pair']).sum().reset_index()
    total_volume['Weightage'] = total_volume['Volume'] / total_volume['Volume'].sum()
    total_volume['Weighted_MLI'] = total_volume['MLI'] * total_volume['Weightage']
    overall_mli = total_volume['Weighted_MLI'].sum()
    print(f"Overall MLI for Coinbase Exchange: {overall_mli}")

    # Save overall MLI to a CSV file in the destination folder
    total_volume.to_csv(os.path.join(data_folder, "overall_mli.csv"), index=False)
    print("Saved Overall MLI to CSV")
else:
    print("No MLI values calculated due to errors")


Downloaded data for XLM-USDT
MLI for XLM-USDT: 1.0657250862887453e-08
Downloaded data for OXT-USD
MLI for OXT-USD: 1.6843722843929257e-09
Downloaded data for PAX-USD
MLI for PAX-USD: 3.515397988622165e-07
Downloaded data for TRIBE-USD
MLI for TRIBE-USD: 3.3100191764725436e-07
Downloaded data for MATIC-GBP
MLI for MATIC-GBP: 6.789603615669372e-07
Downloaded data for MANA-EUR
MLI for MANA-EUR: 1.7064187318119054e-06
Downloaded data for AUCTION-USDT
MLI for AUCTION-USDT: 0.03462228412340046
Downloaded data for RARE-USD
MLI for RARE-USD: 3.96703585317342e-08
Downloaded data for AXS-EUR
MLI for AXS-EUR: 0.061304353093667004
Downloaded data for JASMY-USDT
MLI for JASMY-USDT: 1.9008581330117768e-11
Downloaded data for CVC-USDC
MLI for CVC-USDC: 6.570770501119224e-08
Downloaded data for LOOM-USDC
MLI for LOOM-USDC: 1.1121783990224892e-08
Downloaded data for ROSE-USDT
MLI for ROSE-USDT: 1.0366303764038431e-08
Downloaded data for BUSD-USD
MLI for BUSD-USD: 1.162348997726072e-09
Downloaded data f

#Download the calculated liquidities for 300 day period.

In [2]:
from google.colab import files

# Download individual_mli.csv file
files.download('/content/coinbase_data/individual_mli.csv')

# Download overall_mli.csv file
files.download('/content/coinbase_data/overall_mli.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Now we need to calcualte liquidity for all available period. ( 5 years in our case). 
UML:
```mermaid
sequenceDiagram
participant Client as Client
participant API as Coinbase API
participant CSV as CSV Files

Client->>API: Request list of currency pairs
API->>Client: Return list of currency pairs

loop for each currency pair
    Client->>API: Request historical OHLCV data (start_date, granularity)
    API->>Client: Return historical OHLCV data
    Client->>Client: Calculate MLI for each currency pair
    Client->>CSV: Save individual OHLCV data and MLI to CSV files
end

Client->>Client: Calculate overall exchange MLI
Client->>CSV: Save overall exchange MLI to CSV file
```

**Download 5 years data using pagination technique**

## **FIXING DATA ISSUES** 
*Only the below code is correct as it is having all historical data for all coins available on coinbase* 

In [1]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
import time

# Define the API endpoint
url = 'https://api.pro.coinbase.com'

# Define the path where the CSV files will be stored
data_folder = '/content/coinbase_data'

# Create the data folder if it doesn't exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

all_ml_values = []

# Define the function to download data for a given currency pair
def download_data(pair_id):
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    start_date = end_date - timedelta(days=365*5 + 1)  # data for past 5 years

    all_data = []

    # Download the data in batches of 300 records
    # Download the data in batches of 300 records
    while start_date < end_date:
        batch_end_date = start_date + timedelta(days=299)
        
        if batch_end_date >= end_date:
            batch_end_date = end_date - timedelta(days=1)
            
        data_url = f'{url}/products/{pair_id}/candles?start={start_date.date()}&end={batch_end_date.date()}&granularity=86400'
        response = requests.get(data_url)
        if response.status_code == 429:
            print(f"Request limit reached for {pair_id}. Waiting for 1 second...")
            time.sleep(1)
            continue
        if response.status_code != 200:
            print(f"Error downloading data for {pair_id}")
            break
        df = pd.DataFrame(response.json(), columns=['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        if len(df) == 0:
            print(f"No more data available for {pair_id}")
            break
        all_data.extend(df.values.tolist())
        print(f"Downloaded data for {pair_id} from {start_date} to {batch_end_date}")
        start_date = batch_end_date + timedelta(days=1)


# download_data function end

    # Save downloaded data to a CSV file
    if all_data:
        df_all = pd.DataFrame(all_data, columns=['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df_all.to_csv(f'{data_folder}/{pair_id}.csv', index=False)

        # Calculate MLI for the currency pair
        mlis = []
        for i in range(1, len(df_all)):
            pt = df_all.iloc[i]['Close']
            pt_minus_1 = df_all.iloc[i-1]['Close']
            vt = df_all.iloc[i]['Volume']
            mli = ((pt - pt_minus_1) ** 2) / vt
            mlis.append(mli)
        mli_value = sum(mlis)
        volume = df_all['Volume'].sum()
        all_ml_values.append({'Currency Pair': pair_id, 'MLI': mli_value, 'Volume': volume})
        print(f"MLI for {pair_id}: {mli_value}")

# Get a list of all currency pairs
pairs = requests.get(f'{url}/products').json()

# Download data and calculate MLI for all currency pairs
for pair in pairs:
    download_data(pair['id'])

# Save the individual MLI values to a CSV file in the destination folder
if all_ml_values:
    pd.DataFrame(all_ml_values).to_csv(f'{data_folder}/individual_mli.csv', index=False)
    print('Saved individual MLI values to CSV')
# Calculate overall exchange MLI weighted with traded volume
if all_ml_values:
    df_all = pd.DataFrame(all_ml_values)
    total_volume = df_all.groupby(['Currency Pair']).sum().reset_index()
    total_volume['Weightage'] = total_volume['Volume'] / total_volume['Volume'].sum()
    total_volume['Weighted_MLI'] = total_volume['MLI'] * total_volume['Weightage']
    overall_mli = total_volume['Weighted_MLI'].sum()
    print(f"Overall MLI for Coinbase Exchange: {overall_mli}")
else:
    print("No MLI values calculated due to errors")

No more data available for MATIC-USD
No more data available for VGX-EUR
No more data available for LINK-USDT
No more data available for DOGE-BTC
No more data available for PYR-USD
No more data available for UMA-USD
Downloaded data for ETC-BTC from 2018-04-05 00:00:00 to 2019-01-29 00:00:00
Downloaded data for ETC-BTC from 2019-01-30 00:00:00 to 2019-11-25 00:00:00
Downloaded data for ETC-BTC from 2019-11-26 00:00:00 to 2020-09-20 00:00:00
Downloaded data for ETC-BTC from 2020-09-21 00:00:00 to 2021-07-17 00:00:00
Downloaded data for ETC-BTC from 2021-07-18 00:00:00 to 2022-05-13 00:00:00
Downloaded data for ETC-BTC from 2022-05-14 00:00:00 to 2023-03-09 00:00:00
Downloaded data for ETC-BTC from 2023-03-10 00:00:00 to 2023-04-04 00:00:00
MLI for ETC-BTC: 1.2868877628565583e-09
No more data available for WBTC-BTC
No more data available for REQ-USDT
No more data available for AAVE-EUR
No more data available for SUPER-USD
No more data available for SHPING-USD
No more data available for CEL

In [2]:
# Save overall MLI to a CSV file in the destination folder
total_volume.to_csv(os.path.join(data_folder, "overall_mli.csv"), index=False)
print("Saved Overall MLI to CSV")


Saved Overall MLI to CSV


In [3]:
from google.colab import files

# Download individual_mli.csv file
files.download('/content/coinbase_data/individual_mli.csv')

# Download overall_mli.csv file
files.download('/content/coinbase_data/overall_mli.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import zipfile
import os
from google.colab import files

output_zipfile = '/content/coinbase_data.zip'

with zipfile.ZipFile(output_zipfile, 'w') as zf:
    for root, _, files in os.walk('/content/coinbase_data'):
        for file in files:
            zf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), '/content/coinbase_data'))



In [7]:
import shutil
from google.colab import files

output_zipfile = '/content/coinbase_data.zip'

# Create a zip file for the data folder
shutil.make_archive('/content/coinbase_data', 'zip', data_folder)

# Download the zip file in Google Colab
files.download(output_zipfile)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>